In [1]:
pip install boto3 opencv-python cvzone mediapipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import cv2
from cvzone.HandTrackingModule import HandDetector

In [10]:
def launch_instance():
    myec2 = boto3.client("ec2")
    print(myec2.run_instances(
        ImageId='ami-0ded8326293d3201b',
        InstanceType="t2.micro",
        MaxCount=1,
        MinCount=1
    ))

def stop_instance():
    myec2 = boto3.client("ec2")
    instance_name = "new-instance"
    print(myec2.stop_instances(
        Filters=[
            {
                'Name': 'tag:Name',
                'Values': [instance_name]
            }
        ]
    ))

def list_instances():
    myec2 = boto3.client("ec2")
    print(myec2.describe_instances())

In [ ]:
cam = cv2.VideoCapture(0)

while True:
    success, img = cam.read()
    cv2.imshow("Image", img)

    if cv2.waitKey(100) == ord('q'):
        break

    detect = HandDetector()
    handpic = detect.findHands(img, draw=False)

    if handpic:
        landmark_list = handpic[0]
        finger_status = detect.fingersUp(landmark_list)

        if finger_status == [1, 1, 1, 1, 1] and landmark_list['type'] == 'Left':
            break

        if finger_status == [1, 0, 0, 0, 0] and landmark_list['type'] == 'Left':
            list_instances()

        if finger_status and landmark_list['type'] == 'Right':
            for i in finger_status:
                if i == 1:
                    print(launch_instance())
        
cv2.destroyAllWindows()
cam.release()

{'Groups': [], 'Instances': [{'AmiLaunchIndex': 0, 'ImageId': 'ami-0ded8326293d3201b', 'InstanceId': 'i-0b09b7752ea02d22d', 'InstanceType': 't2.micro', 'LaunchTime': datetime.datetime(2023, 9, 28, 17, 12, 54, tzinfo=tzutc()), 'Monitoring': {'State': 'disabled'}, 'Placement': {'AvailabilityZone': 'ap-south-1a', 'GroupName': '', 'Tenancy': 'default'}, 'PrivateDnsName': 'ip-172-31-36-197.ap-south-1.compute.internal', 'PrivateIpAddress': '172.31.36.197', 'ProductCodes': [], 'PublicDnsName': '', 'State': {'Code': 0, 'Name': 'pending'}, 'StateTransitionReason': '', 'SubnetId': 'subnet-0571dd0363eab3fe9', 'VpcId': 'vpc-0626a9bb8e46232e1', 'Architecture': 'x86_64', 'BlockDeviceMappings': [], 'ClientToken': '4dc6513b-149b-4458-9bc3-a9ca7041b7be', 'EbsOptimized': False, 'EnaSupport': True, 'Hypervisor': 'xen', 'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2023, 9, 28, 17, 12, 54, tzinfo=tzutc()), 'AttachmentId': 'eni-attach-00fbbc59e72145f8b', 'DeleteOnTermination': True,

: 